컨텐츠 기반 영화추천

입력 데이터 로딩

In [42]:
import pandas as pd
import numpy as np


In [43]:
movies = pd.read_csv("http://grepp-reco-test.s3.ap-northeast-2.amazonaws.com/tmdb_5000_movies.csv")

In [44]:
#장르 이름만 빼와서 장르 스트링을 만든다
import json

#장르네임 칼럼을 새로 만들거고 [{"id": 12, "name": "Adventure"}, {"id": 14, ".. 이렇게 표시되어있는 형태에서
# adventure 같은 name 필드에 있는 값만 가져올것이다

def add_genre_name(j):
    genres=[]
    ar = json.loads(j)
    for a in ar:
        genres.append(a.get("name"))
    return " ".join(sorted(genres))

#위에서 만든 장르이름만 추출한 것을 기존 movies에 장르 열을 만들어 붙인다
movies[' genres_name '] = movies.apply(lambda x: add_genre_name(x.genres), axis=1) 

이제 
여러 텍스트 필드들을 모아 텍스트 유사도에 사용할 텍스트 필드 하나를 생성해보자

In [45]:
for f in ['original_title', 'overview',' genres_name ']:
    movies[f] = movies[f].fillna('') #값이 없는 na를 그냥 냅두면 error발생하니까 빈 문자열로 바꿔준다

In [46]:
def combine_features(row):
    try:
        return row['original_title']+" "+ row['overview']+" "+ row[' genres_name ']
    except:
        print ("Error:", row)


In [47]:
movies["combined_features"]= movies.apply(combine_features, axis=1)
#람다 func로 위의 함수에서 만들어준 새로운 열 만들어줌


movies = movies.reset_index()

In [48]:
movies["combined_features"].head()

0    Avatar In the 22nd century, a paraplegic Marin...
1    Pirates of the Caribbean: At World's End Capta...
2    Spectre A cryptic message from Bond’s past sen...
3    The Dark Knight Rises Following the death of D...
4    John Carter John Carter is a war-weary, former...
Name: combined_features, dtype: object

Tf-idf 기반 벡터 생성 후 코사인 유사도로 영화들간의 유사도 계산

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [51]:
TfidfVectorizer = TfidfVectorizer(analyzer = 'word', stop_words = 'english',norm ='l2')

In [52]:
tfidf_matrix = TfidfVectorizer.fit_transform(movies["combined_features"]) #단어단위로 바뀐 행렬이 리턴

In [53]:
tfidf_matrix.shape #열이 엄청 늘었음 -> 굉장히 많은 단어들이 발견되었다

(4803, 22179)

In [54]:
cosine_sim = cosine_similarity(tfidf_matrix)

In [55]:
df_cosine_sim=pd.DataFrame(data= cosine_sim)
df_cosine_sim.head()

,0,1,2,3,4,5,6,7,8,9,...,4793,4794,4795,4796,4797,4798,4799,4800,4801,4802
0,1.000000,0.034109,0.013909,0.026299,0.029273,0.046984,0.003855,0.068451,0.020614,0.020293,...,0.000000,0.000000,0.026697,0.057602,0.000000,0.003941,0.0,0.000000,0.000000,0.000000
1,0.034109,1.000000,0.014576,0.004304,0.047903,0.023093,0.004040,0.039814,0.021603,0.029597,...,0.016368,0.044292,0.006000,0.000000,0.000000,0.004130,0.0,0.022717,0.016449,0.000000
2,0.013909,0.014576,1.000000,0.008018,0.009303,0.009417,0.003084,0.041851,0.023712,0.008672,...,0.022647,0.000000,0.000000,0.000000,0.015928,0.007694,0.0,0.011636,0.000000,0.000000
3,0.026299,0.004304,0.008018,1.000000,0.011748,0.007161,0.014069,0.027766,0.028148,0.158541,...,0.002655,0.003020,0.001183,0.004516,0.001678,0.007343,0.0,0.029067,0.038273,0.019549
4,0.029273,0.047903,0.009303,0.011748,1.000000,0.007872,0.011027,0.054503,0.005454,0.019947,...,0.010447,0.000000,0.000000,0.013190,0.000000,0.002636,0.0,0.004744,0.000000,0.000000


컨텐츠 기반 추천함수 만들기

In [56]:
def get_title_from_index(df, index):
    return df[df.index == index]["original_title"].values[0]
def get_index_from_title(df, title):
    return df[df.original_title==title]["index"].values[0]

#위의 데이터 프레임은 인덱스 기반으로 만들어져 있기 때문에 movie tite과 같은 걸로는 찾을 수 없다
#따라서 이거의 인덱스 값을 구하는 함수를 만들어 준거임

In [ ]:
#enumerate 돌려보기
for cs in enumerate(cosine_sim[0]):
    print(cs) 

#인덱스 0인 첫번째 행이 리턴됨 
#enumerate안쓰면 인덱스 값 없이 출력되어서 어디 값인지 찾기 힘들다

In [62]:
def reco_top_similar_movies(movie_title, n=10):
    #movie title 에 해당하는 인덱스를 앞서 작성한 함수를 사용해 가져온다
    movie_index = get_index_from_title(movies, movie_title)
   
    #그 무비 인덱스를 가지고 유사도가 계산된 행렬(cosine_sim)에서 해당 유사도 값들을 가져오고 enumerate를 써서 일련번호를 붙여
    # 그 값들을 저장함 #일련번호,유사도 값으로 저장됨
    similar_movies= enumerate(cosine_sim[movie_index])
    
    #sort기준을 안써주면 걍 일련번호로 되는데 우리가 원하는건 유사도 기준으로 정렬하는 거라 lamda써서 두번째(인덱스 1)로 sort 기준을 지정해준다
    sorted_similar_movies = sorted(similar_movies, key=lambda x :x[1], reverse = True)


    ret_movies =[]
    i=0
    #유사도 기준으로 정렬된 리스트들을 맨 위부터 차례대로 뒤지며 요청된 숫자만큼 찾아서 
    #해당하는 인덱스를 무비 타이틀로 바꿔서 리턴해줌
    for element in sorted_similar_movies:
        title = get_title_from_index(movies, element[0])
        ret_movies.append(title)
        i = i+1
        if i >= n:
            break
    return ret_movies

In [63]:
print(reco_top_similar_movies('Avatar',5))

['Avatar', 'Apollo 18', 'The American', 'Obitaemyy Ostrov', 'The Matrix']


In [64]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [65]:
!jupyter nbconvert --to html "/content/drive/MyDrive/Colab Notebooks/content_recommendation.ipynb"

[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/content_recommendation.ipynb to html
[NbConvertApp] Writing 298365 bytes to /content/drive/MyDrive/Colab Notebooks/content_recommendation.html
